In [1]:

import pandas as pd 
import mysql.connector

conn_mysql = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Praveen@1205",
    database='amazon'
)

cursor_mysql = conn_mysql.cursor()
print("MySQL connection established!")




MySQL connection established!


In [2]:
df=pd.read_csv("E:/PROJECT_02/data_set/amazon_india_2015.csv")

C:\Users\Praveen\AppData\Local\Temp\ipykernel_5544\1059972390.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("E:/PROJECT_02/data_set/amazon_india_2015.csv")


In [26]:
df.keys()

Index(['transaction_id', 'order_date', 'customer_id', 'product_id',
       'product_name', 'category', 'subcategory', 'brand',
       'original_price_inr', 'discount_percent', 'discounted_price_inr',
       'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr',
       'customer_city', 'customer_state', 'customer_tier',
       'customer_spending_tier', 'customer_age_group', 'payment_method',
       'delivery_days', 'delivery_type', 'is_prime_member', 'is_festival_sale',
       'festival_name', 'customer_rating', 'return_status', 'order_month',
       'order_year', 'order_quarter', 'product_weight_kg', 'is_prime_eligible',
       'product_rating', 'order_day'],
      dtype='object')

In [24]:
df['order_date'] = pd.to_datetime(df['order_date'])

In [30]:
df['order_day']=df['order_date'].dt.day
df.to_csv("E:/PROJECT_02/data_set/amazon_india_complete_2015_2025.csv", index=False)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121999 entries, 0 to 1121998
Data columns (total 35 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   transaction_id          1121999 non-null  object        
 1   order_date              1121999 non-null  datetime64[ns]
 2   customer_id             1121999 non-null  object        
 3   product_id              1121999 non-null  object        
 4   product_name            1121999 non-null  object        
 5   category                1121999 non-null  object        
 6   subcategory             1121999 non-null  object        
 7   brand                   1121999 non-null  object        
 8   original_price_inr      1121999 non-null  float64       
 9   discount_percent        1121999 non-null  float64       
 10  discounted_price_inr    1121999 non-null  float64       
 11  quantity                1121999 non-null  int64         
 12  subtotal_inr  

In [ ]:
# 1. Create PRODUCTS Table
cursor_mysql.execute("""
    CREATE TABLE IF NOT EXISTS products (
        product_id VARCHAR(50) NOT NULL,
        product_name VARCHAR(255),
        category VARCHAR(50),
        subcategory VARCHAR(50),
        brand VARCHAR(50),
        product_weight_kg DECIMAL(10, 2),
        is_prime_eligible BOOLEAN,
        product_rating DECIMAL(10, 1),
        PRIMARY KEY (product_id)
    );
""")
print("products Table created")
conn_mysql.commit()

# 2. Create CUSTOMERS Table
cursor_mysql.execute("""
    CREATE TABLE IF NOT EXISTS customers (
        customer_id VARCHAR(50) NOT NULL,
        customer_city VARCHAR(100),
        customer_state VARCHAR(100),
        customer_tier VARCHAR(50),
        customer_spending_tier VARCHAR(50),
        customer_age_group VARCHAR(50),
        PRIMARY KEY (customer_id)
    );
""")
print("customers Table created")
conn_mysql.commit()

# 3. Create TIME_DIMENSION Table
cursor_mysql.execute("""
    CREATE TABLE IF NOT EXISTS time_dimension (
        order_date DATE NOT NULL,
        order_day INT,
        order_month INT,
        order_year INT,
        order_quarter INT,
        PRIMARY KEY (order_date)
    );
""")
print("time_dimension Table created")
conn_mysql.commit()

# 4. Create TRANSACTIONS Table (The Fact Table)
cursor_mysql.execute("""
    CREATE TABLE IF NOT EXISTS transactions (
        transaction_id VARCHAR(50) NOT NULL,
        order_date DATE NOT NULL,
        customer_id VARCHAR(50) NOT NULL,
        product_id VARCHAR(50) NOT NULL,
        original_price_inr DECIMAL(10, 2),
        discount_percent DECIMAL(10, 2),
        discounted_price_inr DECIMAL(10, 2),
        quantity INT,
        subtotal_inr DECIMAL(10, 2),
        delivery_charges DECIMAL(10, 2), 
        final_amount_inr DECIMAL(10, 2),
        payment_method VARCHAR(50),                     
        delivery_days INT,
        delivery_type VARCHAR(50),              
        is_prime_member BOOLEAN,                
        is_festival_sale BOOLEAN,
        festival_name VARCHAR(50),
        customer_rating DECIMAL(5, 1),                      
        return_status VARCHAR(50),              
        
        PRIMARY KEY (transaction_id),
        
        -- Foreign Key Constraints (These will now succeed!)
        FOREIGN KEY (order_date) REFERENCES time_dimension(order_date),
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
        FOREIGN KEY (product_id) REFERENCES products(product_id)
    );
""")
print("transactions Table created")
conn_mysql.commit()

products Table created
customers Table created
time_dimension Table created
transactions Table created


In [27]:
df.head()

,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,...,festival_name,customer_rating,return_status,order_month,order_year,order_quarter,product_weight_kg,is_prime_eligible,product_rating,order_day
0,TXN_2023_00063013,2023-07-23,CUST_2023_00018393,PROD_000454,Vivo Y95 64GB Black,Electronics,Smartphones,Vivo,27340.8400,21.57,...,NaN,4.0,Delivered,7,2023,3,0.20,True,3.5,23
1,TXN_2021_00064486,2021-07-20,CUST_2015_00002865,PROD_000579,Realme Realme 3 128GB Black,Electronics,Smartphones,Realme,32907.4900,0.00,...,NaN,5.0,Delivered,7,2021,3,0.21,False,4.5,20
2,TXN_2017_00065617,2017-11-16,CUST_2016_00004057,PROD_000295,Vivo V7 32GB Blue,Electronics,Smartphones,Vivo,47052.1800,21.91,...,NaN,5.0,Delivered,11,2017,4,0.24,True,4.3,16
3,TXN_2020_00054393,2020-05-04,CUST_2020_00014574,PROD_001654,Alienware Pavilion 4GB RAM Silver,Electronics,Laptops,Alienware,2387.2544,59.60,...,Summer Sale,5.0,Delivered,5,2020,2,1.85,True,3.6,4
4,TXN_2018_00071646,2018-10-09,CUST_2018_00006275,PROD_000095,Motorola Moto X Play 16GB White,Electronics,Smartphones,Motorola,25970.7600,0.00,...,NaN,4.0,Delivered,10,2018,4,0.16,False,3.7,9


In [19]:
import pandas as pd
import mysql.connector

# =================== MySQL Connection ===================
conn_mysql = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Praveen@1205",
    database='amazon'
)
cursor_mysql = conn_mysql.cursor()
print("✅ MySQL connection established!")

# =================== Load CSV ===================
df = pd.read_csv("E:/PROJECT_02/data_set/amazon_india_complete_2015_2025.csv")

# =================== TIME COLUMNS ===================
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
df['order_day'] = df['order_date'].dt.day
df['order_month'] = df['order_date'].dt.month
df['order_year'] = df['order_date'].dt.year
df['order_quarter'] = df['order_date'].dt.quarter

# =================== Common Batch Function ===================
def batch_insert(records, sql_query, table_name, batch_size=10000):
    total = len(records)
    for i in range(0, total, batch_size):
        batch = records[i:i + batch_size]
        cursor_mysql.executemany(sql_query, batch)
        conn_mysql.commit()
        print(f"✅ Inserted {i + len(batch):,} / {total:,} into {table_name}")

# =================== TIME_DIMENSION ===================
time_records = [
    (row.order_date.strftime('%Y-%m-%d') if pd.notnull(row.order_date) else None,
     row.order_day, row.order_month, row.order_year, row.order_quarter)
    for row in df.itertuples(index=False)
]

time_sql = """
    INSERT IGNORE INTO time_dimension (order_date, order_day, order_month, order_year, order_quarter)
    VALUES (%s, %s, %s, %s, %s)
"""
batch_insert(time_records, time_sql, "time_dimension")

# =================== PRODUCTS ===================
product_records = [
    (row.product_id, row.product_name, row.category, row.subcategory, row.brand,
     row.product_weight_kg, bool(row.is_prime_eligible), row.product_rating)
    for row in df.itertuples(index=False)
]

product_sql = """
    INSERT IGNORE INTO products (product_id, product_name, category, subcategory, brand,
                                 product_weight_kg, is_prime_eligible, product_rating)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""
batch_insert(product_records, product_sql, "products")

# =================== CUSTOMERS ===================
customer_records = [
    (row.customer_id, row.customer_city, row.customer_state, row.customer_tier,
     row.customer_spending_tier, row.customer_age_group)
    for row in df.itertuples(index=False)
]

customer_sql = """
    INSERT IGNORE INTO customers (customer_id, customer_city, customer_state,
                                  customer_tier, customer_spending_tier, customer_age_group)
    VALUES (%s, %s, %s, %s, %s, %s)
"""
batch_insert(customer_records, customer_sql, "customers")

# =================== TRANSACTIONS ===================
transaction_records = [
    (row.transaction_id, row.order_date.strftime('%Y-%m-%d') if pd.notnull(row.order_date) else None,
     row.customer_id, row.product_id, row.original_price_inr, row.discount_percent,
     row.discounted_price_inr, row.quantity, row.subtotal_inr, row.delivery_charges,
     row.final_amount_inr, row.payment_method, row.delivery_days, row.delivery_type,
     bool(row.is_prime_member), bool(row.is_festival_sale), row.festival_name,
     row.customer_rating, row.return_status)
    for row in df.itertuples(index=False)
]

transaction_sql = """
    INSERT IGNORE INTO transactions (
        transaction_id, order_date, customer_id, product_id,
        original_price_inr, discount_percent, discounted_price_inr, quantity,
        subtotal_inr, delivery_charges, final_amount_inr, payment_method,
        delivery_days, delivery_type, is_prime_member, is_festival_sale,
        festival_name, customer_rating, return_status
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
batch_insert(transaction_records, transaction_sql, "transactions")

# =================== Close Connection ===================
cursor_mysql.close()
conn_mysql.close()
print("🎯 All data inserted successfully! Connection closed.")


✅ MySQL connection established!
✅ Inserted 10,000 / 1,121,999 into time_dimension
✅ Inserted 20,000 / 1,121,999 into time_dimension
✅ Inserted 30,000 / 1,121,999 into time_dimension
✅ Inserted 40,000 / 1,121,999 into time_dimension
✅ Inserted 50,000 / 1,121,999 into time_dimension
✅ Inserted 60,000 / 1,121,999 into time_dimension
✅ Inserted 70,000 / 1,121,999 into time_dimension
✅ Inserted 80,000 / 1,121,999 into time_dimension
✅ Inserted 90,000 / 1,121,999 into time_dimension
✅ Inserted 100,000 / 1,121,999 into time_dimension
✅ Inserted 110,000 / 1,121,999 into time_dimension
✅ Inserted 120,000 / 1,121,999 into time_dimension
✅ Inserted 130,000 / 1,121,999 into time_dimension
✅ Inserted 140,000 / 1,121,999 into time_dimension
✅ Inserted 150,000 / 1,121,999 into time_dimension
✅ Inserted 160,000 / 1,121,999 into time_dimension
✅ Inserted 170,000 / 1,121,999 into time_dimension
✅ Inserted 180,000 / 1,121,999 into time_dimension
✅ Inserted 190,000 / 1,121,999 into time_dimension
✅ Insert